In [1]:
%load_ext watermark
%watermark -v -p numpy,scipy,sklearn,pandas,matplotlib

CPython 3.6.5
IPython 4.0.0

numpy 1.14.5
scipy 1.1.0
sklearn 0.19.2
pandas 0.23.3
matplotlib 2.2.2


In [2]:
# 파이썬 2와 파이썬 3 지원
from __future__ import division, print_function, unicode_literals

# 공통
import numpy as np
import os
from datetime import datetime, date, time

# 일관된 출력을 위해 유사난수 초기화
np.random.seed(42)

# 맷플롯립 설정
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12


In [3]:
from __future__ import division, print_function, unicode_literals
import tensorflow as tf
import pandas as pd
from datetime import datetime, date, time

In [4]:
# MNIST 데이터 임포트
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
# Define some parameters
element_size = 28
time_steps = 28
num_classes = 10
batch_size = 128
hidden_layer_size = 128


# 텐서보드 모델 요약을 저장할 위치
LOG_DIR = "logs/RNN_with_summaries_"+datetime.now().strftime('%Y%m%d%H%M%S')+"/"

# Create placeholders for inputs, labels
_inputs = tf.placeholder(tf.float32,
                         shape=[None, time_steps, element_size],
                         name='inputs')
y = tf.placeholder(tf.float32, shape=[None, num_classes], name='labels')
_inputs.get_shape(), y.get_shape()


(TensorShape([Dimension(None), Dimension(28), Dimension(28)]),
 TensorShape([Dimension(None), Dimension(10)]))

## RNN 단계

In [6]:
# This helper function taken from official TensorFlow documentation,
# simply add some ops that take care of logging summaries
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

In [7]:
def rnn_step(previous_hidden_state, x):
    current_hidden_state = tf.tanh(tf.matmul(previous_hidden_state, Wh) + tf.matmul(x, Wx) + b_rnn)
#     print('previous_hidden_state {}'.format(previous_hidden_state))
#     print('Wh {}'.format(Wh))
#     print('x {}'.format(x))
#     print('Wx {}'.format(Wx))
#     print('b_rnn {}'.format(b_rnn))
#     hh = tf.matmul(previous_hidden_state, Wh)
#     print('hh {}'.format(hh))
#     xx = tf.matmul(x, Wx)
#     print('xx {}'.format(xx))
#     multi_1 = hh + xx
#     print('multi_1 {}'.format(multi_1))
    
    return current_hidden_state



In [8]:
# Weights and bias for input and hidden layer
with tf.name_scope('rnn_weights'):
    with tf.name_scope("W_x"):
        Wx = tf.Variable(tf.zeros([element_size, hidden_layer_size]))
        variable_summaries(Wx)
    with tf.name_scope("W_h"):
        Wh = tf.Variable(tf.zeros([hidden_layer_size, hidden_layer_size]))
        variable_summaries(Wh)
    with tf.name_scope("Bias"):
        b_rnn = tf.Variable(tf.zeros([hidden_layer_size]))
        variable_summaries(b_rnn)

In [9]:
# Processing inputs to work with scan function
# Current input shape: (batch_size, time_steps, element_size)
processed_input = tf.transpose(_inputs, perm=[1, 0, 2])
# Current input shape now: (time_steps,batch_size, element_size)


initial_hidden = tf.zeros([batch_size, hidden_layer_size])
# Getting all state vectors across time
all_hidden_states = tf.scan(rnn_step,
                            processed_input,
                            initializer=initial_hidden,
                            name='states')
all_hidden_states
initial_hidden

<tf.Tensor 'zeros:0' shape=(128, 128) dtype=float32>

In [10]:
print(_inputs)
print(processed_input)
print(initial_hidden)
print(all_hidden_states)

Tensor("inputs:0", shape=(?, 28, 28), dtype=float32)
Tensor("transpose:0", shape=(28, ?, 28), dtype=float32)
Tensor("zeros:0", shape=(128, 128), dtype=float32)
Tensor("states/TensorArrayStack/TensorArrayGatherV3:0", shape=(28, 128, 128), dtype=float32)


In [11]:
#%matplotlib inline
#import matplotlib
#import matplotlib.pyplot as plt
#batch_size = 2
#batch_x, batch_y = mnist.train.next_batch(batch_size)

#some_digit = batch_x[0]
#some_digit_image = some_digit.reshape(28,28)
#plt.imshow(some_digit_image, cmap = matplotlib.cm.binary, interpolation="nearest")
#plt.axis("off")
#plt.show()
#some_digit = batch_x[1]
#some_digit_image = some_digit.reshape(28,28)
#plt.imshow(some_digit_image, cmap = matplotlib.cm.binary, interpolation="nearest")
#plt.axis("off")
#plt.show()

In [12]:
# batch_size=100
# sess = tf.Session()
# batch_x, batch_y = mnist.train.next_batch(batch_size)
# batch_x = batch_x.reshape((batch_size, time_steps, element_size))
# sess.run(tf.global_variables_initializer())
# out_inputs, out_y, out_processed_input, out_all_hidden_states = sess.run([_inputs, y, processed_input, all_hidden_states],feed_dict={_inputs: batch_x, y: batch_y})
# out_all_hidden_states.shape


In [13]:

# Weights for output layers
with tf.name_scope('linear_layer_weights') as scope:
    with tf.name_scope("W_linear"):
        Wl = tf.Variable(tf.truncated_normal([hidden_layer_size, num_classes],
                                             mean=0, stddev=.01))
        variable_summaries(Wl)
    with tf.name_scope("Bias_linear"):
        bl = tf.Variable(tf.truncated_normal([num_classes],
                                             mean=0, stddev=.01))
        variable_summaries(bl)


In [14]:
# Apply linear layer to state vector
def get_linear_layer(hidden_state):

    return tf.matmul(hidden_state, Wl) + bl

In [15]:
with tf.name_scope('linear_layer_weights') as scope:
    # Iterate across time, apply linear layer to all RNN outputs
    all_outputs = tf.map_fn(get_linear_layer, all_hidden_states)
    # Get Last output -- h_28
    output = all_outputs[-1]
    tf.summary.histogram('outputs', output)
#all_hidden_states
#all_outputs
all_outputs.get_shape(), output.get_shape()

(TensorShape([Dimension(28), Dimension(128), Dimension(10)]),
 TensorShape([Dimension(128), Dimension(10)]))

In [16]:
# batch_size=100
# sess = tf.Session()
# batch_x, batch_y = mnist.train.next_batch(batch_size)
# batch_x = batch_x.reshape((batch_size, time_steps, element_size))
# sess.run(tf.global_variables_initializer())
# outs = sess.run([_inputs, y, processed_input, all_hidden_states, Wl, bl, all_outputs],feed_dict={_inputs: batch_x, y: batch_y})
# outs[3].shape, outs[4].shape, outs[5].shape, outs[6].shape
# outs[5]

In [17]:
with tf.name_scope('linear_layer_weights') as scope:
    # Iterate across time, apply linear layer to all RNN outputs
    all_outputs = tf.map_fn(get_linear_layer, all_hidden_states)
    # Get Last output -- h_28
    output = all_outputs[-1]
    tf.summary.histogram('outputs', output)

with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=y))
    tf.summary.scalar('cross_entropy', cross_entropy)

with tf.name_scope('train'):
    # Using RMSPropOptimizer
    train_step = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cross_entropy)

with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(output, 1))

    accuracy = (tf.reduce_mean(tf.cast(correct_prediction, tf.float32)))*100
    tf.summary.scalar('accuracy', accuracy)

    
# Merge all the summaries
merged = tf.summary.merge_all()


# Get a small test set
test_data = mnist.test.images[:batch_size].reshape((-1, time_steps, element_size))
test_label = mnist.test.labels[:batch_size]

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [18]:

with tf.Session() as sess:
    # Write summaries to LOG_DIR -- used by TensorBoard
    train_writer = tf.summary.FileWriter(LOG_DIR + '/train',
                                         graph=tf.get_default_graph())
    test_writer = tf.summary.FileWriter(LOG_DIR + '/test',
                                        graph=tf.get_default_graph())

    sess.run(tf.global_variables_initializer())
    for i in range(2000):

            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Reshape data to get 28 sequences of 28 pixels
            batch_x = batch_x.reshape((batch_size, time_steps, element_size))
            summary, _ = sess.run([merged, train_step], feed_dict={_inputs: batch_x, y: batch_y})
            # Add to summaries
            train_writer.add_summary(summary, i)

            if i % 1000 == 0:
                acc, loss, = sess.run([accuracy, cross_entropy],
                                      feed_dict={_inputs: batch_x,
                                                 y: batch_y})
                print("Iter " + str(i) + ", Minibatch Loss= " +
                      "{:.6f}".format(loss) + ", Training Accuracy= " +
                      "{:.5f}".format(acc))
            if i % 100 == 0:
                # Calculate accuracy for 128 mnist test images and
                # add to summaries
                summary, acc = sess.run([merged, accuracy],
                                        feed_dict={_inputs: test_data, y: test_label})
                test_writer.add_summary(summary, i)

                test_acc = sess.run(accuracy, feed_dict={_inputs: test_data,
                                                             y: test_label})
                print("Test Accuracy:", test_acc)
print("Complete")

Iter 0, Minibatch Loss= 2.302413, Training Accuracy= 13.28125
Test Accuracy: 7.8125
Test Accuracy: 11.71875
Test Accuracy: 28.125
Test Accuracy: 25.78125
Test Accuracy: 30.46875
Test Accuracy: 52.34375
Test Accuracy: 57.03125
Test Accuracy: 64.0625
Test Accuracy: 70.3125
Test Accuracy: 70.3125
Iter 1000, Minibatch Loss= 0.873824, Training Accuracy= 70.31250
Test Accuracy: 72.65625
Test Accuracy: 78.125
Test Accuracy: 83.59375
Test Accuracy: 83.59375
Test Accuracy: 87.5
Test Accuracy: 84.375
Test Accuracy: 82.8125
Test Accuracy: 63.28125
Test Accuracy: 79.6875
Test Accuracy: 88.28125
Complete
